In [4]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib.pyplot as plt # matplotlib package for figure plotting
import numpy as np # numpy package for array handling
import pandas as pd # pandas package for dataframe handling
import trackpy as tp # trackpy itself
import tifffile as tiff # tifffile package for .tif file handling
import os # os package for directory and file handling

%matplotlib inline

In [5]:
def particle_count(img_path):
    img = tiff.imread(img_path)
    f = tp.locate(img[0], 21, invert=False)
    # f.head  # show a few rows of data
    # get locations of particle in the first frame
    fig_location = tp.annotate(f, img[0]).get_figure()
    fig_location.savefig(img_path.replace('.tif', '_location.png'))
    
    # mass distribution 
    # fig, ax = plt.subplots()
    # ax.hist(f['mass'], bins=20)
    # Optionally, label the axes.
    # ax.set(xlabel='mass', ylabel='count');
    # fig.savefig(img_path.replace('.tif', '_mass.tif'))
    # f = tp.locate(img[0], 23, invert=False, minmass=20) # threshold the min mass
    
    # calculate subpixel accuracy (expecting even distribution)
    # subpx_bias_img = tp.subpx_bias(f)
    # subpx_bias_img.savefig(img_path.replace('.tif', '_subpx'))
    
    f = tp.batch(img[:], 21, minmass=20, invert=False);
    t = tp.link(f, 5, memory=3) # The maximal movement of the particle, memory is the maximal frames allowed for missing particle
    # t.head()
    t1 = tp.filter_stubs(t, 25) # Filter the by the minimal frames of a single track - set to 25 frames
    # Compare the number of particles in the unfiltered and filtered data.
    print('Before:', t['particle'].nunique())
    print('After:', t1['particle'].nunique())
    num_of_particle = t1['particle'].nunique()
    
    if num_of_particle != 0:
        # Plot figure with mass against size
        fig_massTosize = tp.mass_size(t1.groupby('particle').mean()).get_figure() # convenience function -- just plots size vs. mass
        fig_massTosize.savefig(img_path.replace('.tif', '_massTosize.png'))

        # Plot trajaectories figure
        fig_traj = tp.plot_traj(t1).get_figure()
        fig_traj.savefig(img_path.replace('.tif', '_traj.png'))
    
    return num_of_particle, t1['mass'].mean(), t1['size'].mean()

In [6]:
def run_folder(folder_path):
    output = {}
    for r, d, f in os.walk(folder_path):
        for name in f:
            if name.endswith('.tif'):
                sample_path = os.path.join(r, name)
                try:
                    output[name.replace('_MMStack_Pos0.ome.tif', '')] = particle_count(sample_path)
                    print(output)
                except AttributeError or ValueError:
                    output[name.replace('_MMStack_Pos0.ome.tif', '')] = [0,0,0]
                    
    df = pd.DataFrame.from_dict(output, orient ='index')
    df = df.reset_index()
    df.columns = ['name', 'count', 'mean mass', 'mean size']
    print(df)
    df.to_csv(folder_path + '/Summary.csv')

In [ ]:
path = "/Volumes/Backup_Disk/CPGS_data/DOPC/AggConc"
for root, dirs, files in os.walk(path):
    for name in dirs:
        run_folder(os.path.join(root, name))

In [ ]:
path = "/Volumes/Backup_Disk/CPGS_data/POPC"
df = pd.DataFrame()
for root, dirs, files in os.walk(path):
    for name in files:
        if name.endswith(".csv"):
            dff = pd.read_csv(os.path.join(root, name))
            df = pd.concat([df, dff])
df.to_csv(path + '/Summary.csv')

In [8]:
path = "/Volumes/Backup_Disk/CPGS_data/DOPC/Labelled"
run_folder(path)

Frame 299: 182 trajectories present.
Before: 43383
After: 0
{'20191106_1%_2uM_ab_641_2': (303, 307967.8102837104, 4.47201329071633), '20191106_1%_2uM_blank_1': (277, 249356.61307057278, 4.824252415558049), '20191106_1%_2uM_blank_2': (261, 222751.1800296232, 4.904516858840342), '20191106_1%_2uM_ab_641_1': (196, 260990.97331262467, 4.840174252176646), '20191106_0%_2uM_ab_641_2': (129, 313008.2815687244, 4.938123300544988), '20191106_0%_2uM_ab_641_1': (120, 306608.4019317261, 5.126074966071061), '20191106_0%_2uM_blank_1': (0, nan, nan), '20191106_0%_2uM_blank_2': (0, nan, nan)}
                       name  count      mean mass  mean size
0  20191106_1%_2uM_ab_641_2    303  307967.810284   4.472013
1   20191106_1%_2uM_blank_1    277  249356.613071   4.824252
2   20191106_1%_2uM_blank_2    261  222751.180030   4.904517
3  20191106_1%_2uM_ab_641_1    196  260990.973313   4.840174
4  20191106_0%_2uM_ab_641_2    129  313008.281569   4.938123
5  20191106_0%_2uM_ab_641_1    120  306608.401932   